In [1]:
import pandas as pd
import numpy as np
import time as tm
#from numpy import RankWarning
import math
import random
import warnings
import statsmodels.api as sm
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from scipy.stats import rankdata
import scipy.stats as stats
from scipy.stats import genpareto, norm, poisson, expon, gamma
from scipy.special import inv_boxcox
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
from matplotlib.ticker import LogFormatter 
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.lines import Line2D
import seaborn as sns
from Trend import *
from NS_Cluster import *
from scipy.fftpack import fft, ifft
!pip install copulas
!pip install torch_geometric
import copulas.multivariate
from statsmodels.tsa.arima.model import ARIMA
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import TransformerConv
import matplotlib.pyplot as plt
import torch.optim as optim
from torch_geometric.nn import GCNConv  # Graph Neural Network (GNN) layer
from sklearn.preprocessing import MinMaxScaler
import math
!pip install captum
from captum.attr import IntegratedGradients
from scipy.fftpack import fft, ifft
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import chi2
from matplotlib import gridspec
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter
from collections import Counter
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
from numpy.linalg import LinAlgError
import os

  Using cached copulas-0.12.3-py3-none-any.whl.metadata (9.5 kB)
  Using cached plotly-6.2.0-py3-none-any.whl.metadata (8.5 kB)
Using cached copulas-0.12.3-py3-none-any.whl (52 kB)
Using cached plotly-6.2.0-py3-none-any.whl (9.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [copulas]
  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
Using cached torch_geometric-2.6.1-py3-none-any.whl (1.1 MB)
  Using cached captum-0.8.0-py3-none-any.whl.metadata (26 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached captum-0.8.0-py3-none-any.whl (1.4 MB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6━━━━━━ 0/2 [numpy]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [captum]2m1/2 [captum]
ERROR: pip's dependency res

# Function Definitions

In [2]:
def wavelet(Y, dj=0.025):
    """
    Perform a Continuous Wavelet Transform (CWT) on a time series using the Morlet wavelet, calculating its power spectrum.

    :param Y: NumPy array of the original time series data to be analyzed.
    :param dj: Float, optional, specifying the spacing between discrete scales for the wavelet transform. Defaults to 0.025.
    :return: A dictionary containing the wavelet transform ('wave'), associated periods ('period'), scales used ('scale'), wavelet power spectrum ('power'), cone of influence ('coi'), and average power across all times for each scale ('avg_power').
    """
    
    DT = 1/12  # Timestep for monthly data
    pad = 1  # Pad data
    param = 6  # Wavenumber for Morlet
    s0 = DT  # Standard smallest resolvable scale
    n1 = len(Y)  # Total observations
    J1 = int(np.floor((np.log2(n1 * DT / s0)) / dj)) # Equation 10

    # Center and optionally pad the time series to nearest base 2 power
    x = (Y - np.mean(Y))
    if pad == 1:
        base2 = int(np.trunc(np.log(n1) / np.log(2) + 0.4999))
        x = np.concatenate((x, np.zeros(2 ** (base2 + 1) - n1)))

    n = len(x)  # Length with pad, nearest base 2 power

    # Construct wavenumber array
    k = np.arange(1, int(n / 2) + 1)
    k = k * ((2 * np.pi) / (n * DT))
    k = np.concatenate(([0], k, -np.flip(k[:int((n - 1) / 2)])))

    # Compute FFT of the (padded) time series
    f = fft(x)

    # Construct scale array and initialize wave array
    scale = s0 * 2 ** (np.arange(0, J1 + 1) * dj) # Equation 9
    wave = np.zeros((J1 + 1, n), dtype=complex) 

    for a1 in range(J1 + 1):
        scl = scale[a1]
        expnt = -(scl * k - param) ** 2 / (2 * (k > 0))
        norm = np.sqrt(scl * k[1]) * (np.pi ** (-0.25)) * np.sqrt(n)
        daughter = norm * np.exp(expnt) * (k > 0)
        wave[a1, :] = ifft(f * daughter) / n

    fourier_factor = (4 * np.pi) / (param + np.sqrt(2 + param ** 2))
    period = fourier_factor * scale

    # Calculate coi
    lengths = np.concatenate((np.arange(1, np.floor((n1 + 1) / 2) + 1), np.arange(1, np.floor(n1 / 2) + 1)[::-1]))
    coi = fourier_factor / np.sqrt(2) * lengths * DT

    wave = wave[:, :n1]  # Remove padding
    power = (np.abs(wave) ** 2)/np.var(wave) # Calculate normalized power spectrum
    avg_power = np.mean(power, axis=1)

    result = {'wave': wave, 'period': period, 'scale': scale, 'power': power, 'coi': coi, 'avg_power': avg_power}
    return result


def transform_data(time_series, detrended, data_type, plot=False):
    """
    Generates QQ plots for original and detrended data, performs a Box-Cox transformation on the detrended data,
    and plots the QQ plot of the transformed data.
    
    Parameters:
    - time_series: The original time series data.
    - detrended: The detrended time series data.
    - data_type: A string representing the type of data (e.g., "Temperature", "Rainfall").
    
    Returns:
    - A tuple containing the transformed time series and the lambda value of the Box-Cox transformation.
    """

    if plot:
        # QQ Plot for original data versus detrended
        fig, axs = plt.subplots(1, 2, figsize=(10, 5))  # 1 row, 2 columns, and optional figure size
    
        # QQ Plot for original data
        stats.probplot(time_series, dist="norm", plot=axs[0])
        axs[0].set_title(f"QQ Plot of Original {data_type} Data")
        axs[0].set_ylabel("Ordered Values")
        axs[0].set_xlabel("Theoretical Quantiles")
        axs[0].grid(True)
    
        # QQ Plot for detrended data
        stats.probplot(detrended, dist="norm", plot=axs[1])
        axs[1].set_title(f"QQ Plot of Detrended {data_type} Data")
        axs[1].set_ylabel("Ordered Values")
        axs[1].set_xlabel("Theoretical Quantiles")
        axs[1].grid(True)
    
        plt.show()

    # Boxcox transform for detrended data
    shift = abs(min(detrended)) + 1
    timeseries_boxcox, lambda_boxcox = stats.boxcox(detrended + shift)

    if plot:
        # Plot Transform
        stats.probplot(timeseries_boxcox, dist="norm", plot=plt)
        plt.title(f"QQ Plot of Box-Cox Transformed {data_type} Data")
        plt.ylabel("Ordered Values (Transformed)")
        plt.xlabel("Theoretical Quantiles")
        plt.grid(True)
        plt.show()

    return shift, timeseries_boxcox, lambda_boxcox


def CI(wlt, dat, conf, noise_type):
    """
    Calculate the confidence interval for a wavelet power spectrum. Based off of Torrence and Compo 1998.
    
    Parameters:
    - conf: Confidence level as a decimal (0-1).
    - dat: Time series data.
    - noise_type: 'r' for red noise, 'w' for white noise.
    
    Returns:
    - A dictionary with the significant levels ('sig') as the key.
    """
    na = len(dat)
    
    if noise_type == "r":
        model = sm.tsa.ARIMA(dat, order=(1, 0, 0) , trend='n').fit() # prevent intercept term #dat/10^10
        alpha = model.params[0] # Equation 15, extract lag-1 autocorr
        print(f'Red Noise AR1 Coefficient: {alpha}')
    else:
        alpha = 0 # For white noise, default 0
    
    ps = wlt['period']
    LP = len(ps)
    freq = 1 / ps
    
    CI = np.zeros(LP)  # Initialize confidence interval empty array
    
    for i in range(LP):
        P = (1 - (alpha**2)) / (1 + (alpha**2) - (2 * alpha * np.cos(2 * np.pi * freq[i])))  # Equation 16
        df = 2 * np.sqrt(1 + ((na / (2.32 * ps[i]))**2))  # Equation 23, 28, determine dof
        CI[i] = P * (chi2.ppf(conf, df) / df) # Equation 18, 26

    dict = wlt.copy()

    dict['sig'] = CI

    return dict


def reconstruct(WT, series_data, dj=0.025, lvl=0, only_coi=True, only_sig=True, rescale=True):
    """
    Reconstruct the time series from the wavelet transform.

    Parameters:
    WT (dict): Dictionary containing wavelet transform results and metadata.
    lvl (float): Minimum power level for reconstruction.
    only_coi (bool): Whether to use only the cone of influence for reconstruction.
    only_sig (bool): Whether to use only significant parts for reconstruction.
    rescale (bool): Whether to rescale the reconstructed series.

    Returns:
    x_r (np.ndarray): Reconstructed time series.
    """

    Wave = WT['wave']
    Power = WT['power'] 
    Avg_Power = WT['avg_power'] 
    Scale = WT['scale']
    Period = WT['period']
    
    sig = WT['sig']
    dt = 1/12 # for monthly
    nc = len(series_data)
    nr = len(Scale)

    # Prepare reconstruction components
    rec_waves = np.zeros((nr, nc))
    for s_ind in range(nr):
        rec_waves[s_ind, :] = (np.real(Wave[s_ind, :]) / np.sqrt(Scale[s_ind])) * dj * np.sqrt(dt) / (np.pi**(-1/4) * 0.776)

    # Apply filters
    rec_waves = rec_waves * (Power >= lvl)

    if only_sig and sig is not None:
        mask = ~(Avg_Power > sig)  # Identifies the rows where Avg_Power <= sig
        
        # Check if all rows are going to be masked
        if mask.all():
            warnings.warn("All rows will be masked. No significant wavelet periods identified.")
        
        # To get the values in 'Scale' corresponding to non-masked rows in 'rec_waves':
        non_masked_values = np.around(Scale[~mask], decimals=2)
        
        # Print the values from 'Scale' that correspond to non-masked rows
        if non_masked_values.size > 0:
            print("Significant scales:")
            print(non_masked_values)
        else:
            print("No significant scales identified.")
        
        # Mask these rows with np.nan
        rec_waves[mask, :] = np.nan
    
    if only_coi:
        coi = WT['coi']  # Assuming 'coi' is an array of cone of influence values
        for i in range(nc):
            for s_ind in range(nr):
                if Scale[s_ind] > 2**coi[i]:
                    rec_waves[s_ind, i] = np.nan

    # Compute reconstructed series
    x_r = np.nansum(rec_waves, axis=0)

    # Rescale the reconstructed time series
    if rescale:
        x_r = (x_r - np.mean(x_r)) * np.std(series_data) / np.std(x_r) + np.mean(series_data)

    return x_r, non_masked_values
    

def wavelet_plot(plt_dataset, siglvl, name = '', sigtest = 'default'):
    """
    Plot the wavelet power spectrum and the global wavelet spectrum of a time series. This function creates a comprehensive visualization of the wavelet analysis results, facilitating the examination of periodicities in the data and their significance against specified noise models.

    :param plt_dataset: A dictionary containing 'Power', 'Period', 'COI', 'Avg_Power', 'R_noise', and 'W_noise' keys representing the wavelet power spectrum data, periods, cone of influence, average power, and significance levels against red and white noise respectively.
    :param siglvl: A float specifying the significance level for the noise tests (e.g., 0.05 for 95% significance).
    :param sigtest: String, optional, indicating the type of significance test to apply. 'red' for red noise, 'white' for white noise, and 'default' for both. Defaults to 'default'.
    :return: None. The function generates and displays two plots: 1) The wavelet power spectrum, showing power as a function of time and period with areas under the cone of influence shaded. 2) The global wavelet power spectrum, comparing average power over time against significance levels derived from noise models.
    """
    
    fig = plt.figure(figsize=(16, 6))  # Adjusted figure size for horizontal layout

    # Adjust GridSpec for colorbar placement to the left of the first plot with extra space
    gs = gridspec.GridSpec(1, 4, width_ratios=[0.1, 2, 1, 0.1], wspace=0.1)
    
    # Define the start and end colors for your palette
    start_color = '#FFFFFF'
    midpoint_color = '#6699CC'  # White as the neutral midpoint color
    end_color = '#004488'
    
    # Create the custom diverging palette
    custom_palette = sns.blend_palette([start_color, midpoint_color, end_color], as_cmap=True)

    # Wavelet power spectrum subplot
    ax_cwt = fig.add_subplot(gs[1])
    max_power = np.max(plt_dataset['Power'])
    levels = np.linspace(0, max_power, 10)
    
    # Use custom palette for the contourf plot
    time = plt_dataset['Time']
    cs = ax_cwt.contourf(time, np.log2(plt_dataset['Period']), plt_dataset['Power'], 
                         levels=levels, cmap=custom_palette)
    
    ax_cwt.plot(np.arange(time[0], time[0]+len(plt_dataset['COI']), 1), np.log2(plt_dataset['COI']), 'k')
    ax_cwt.fill_between(np.arange(time[0], time[0]+len(plt_dataset['COI']), 1), np.log2(plt_dataset['COI']), np.log2(plt_dataset['Period'][-1]), color='gray', alpha=0.5)
    ax_cwt.set_title(f'Local Wavelet Power Spectrum', fontsize=15)
    ax_cwt.set_ylabel('Period (Years)', fontsize=14)
    ax_cwt.set_xlabel('Time', fontsize=14)
    ax_cwt.tick_params(axis='both', which='major', labelsize=13)
    ax_cwt.set_ylim(np.log2(32), np.log2(2.1))  # Restricting y-axis limits to 2 to 64 years
    ax_cwt.set_yticks(np.log2([4, 8, 16, 32]))
    ax_cwt.set_yticklabels([4, 8, 16, 32])
    
    # Create colorbar for the first subplot
    cbar = plt.colorbar(cs, ax=ax_cwt, orientation='vertical')
    cbar.ax.tick_params(labelsize=13)  # Adjust colorbar tick label size
    # Set the format of the colorbar labels to one decimal place
    cbar.formatter = FormatStrFormatter('%0.1f')
    cbar.update_ticks()
    
    # Global wavelet spectrum subplot
    ax_global = fig.add_subplot(gs[2])
    ax_global.plot(plt_dataset['Avg_Power'], plt_dataset['Period'], label='Power', linestyle='-')
    if sigtest == 'red':
        ax_global.plot(plt_dataset['R_noise'], plt_dataset['Period'], color='red', label=f'Red Noise {int(siglvl*100)}%', linestyle='--')
    elif sigtest == 'white':
        ax_global.plot(plt_dataset['W_noise'], plt_dataset['Period'], color='black', label=f'White Noise {int(siglvl*100)}%', linestyle='--')
    else:
        ax_global.plot(plt_dataset['R_noise'], plt_dataset['Period'], color='red', label=f'Red Noise {int(siglvl*100)}%', linestyle='--')
        ax_global.plot(plt_dataset['W_noise'], plt_dataset['Period'], color='black', label=f'White Noise {int(siglvl*100)}%', linestyle='--')
    ax_global.set_title(f'Global Wavelet Power Spectrum', fontsize=15)
    ax_global.set_xlabel('Power', fontsize=14)
    ax_global.tick_params(axis='both', which='major', labelsize=13)
    ax_global.set_yscale('log', base=2)  # Log scale for y-axis
    ax_global.invert_yaxis()  # To match R's scale_y_continuous(trans = reverselog_trans(2))
    ax_global.set_xlim([0, 5])  # Sets x-axis to range from 0 to 5
    ax_global.set_ylim([32, 2.1])  # Sets y-axis to range from 2 to 64
    
    # Set custom y-axis labels
    y_ticks = [2**x for x in range(2, 6)]  # Up to 128
    ax_global.set_yticks(y_ticks)  # Set y-ticks to be powers of two
    ax_global.set_yticklabels([str(y) for y in y_ticks])  # Use the actual numbers as labels

    fig.suptitle(f'{name} Gauge', fontsize=17)
    plt.legend(fontsize=14)
    plt.show()

In [3]:
def extract_exceedance_clusters(df, signal, var_interest, base_signal):
    # Add Month as Period to daily dataframe
    df['Month'] = df.index.to_period('M')
    
    # Ensure signal index is a PeriodIndex
    signal = signal.copy()
    if not isinstance(signal.index, pd.PeriodIndex):
        if isinstance(signal.index, pd.DatetimeIndex):
            signal.index = signal.index.to_period('M')
        else:
            raise ValueError("Signal index must be a DatetimeIndex or PeriodIndex.")

    # Build monthly signal dictionary directly
    signal_dict = signal[var_interest].to_dict()

    # Map the monthly signal to the daily dataframe
    df['Month_Signal'] = df['Month'].map(signal_dict)

    # Determine if daily values exceed the baseline signal
    df['Exceeds_Std'] = (df[var_interest] > base_signal).astype(int)
    
    # Calculate exceedance difference
    df['Exceedance_Diff_Std'] = np.where(df['Exceeds_Std'] == 1, df[var_interest], 0)
    
    # Identify changes in exceedance state
    df['Exceeds_Change_Std'] = df['Exceeds_Std'].diff()
    
    # Identify the start and end of a cluster
    df['Start_Cluster_Std'] = (df['Exceeds_Change_Std'] == 1)
    df['End_Cluster_Std'] = (df['Exceeds_Change_Std'] == -1)

    # Initialize result list
    result = []

    # Iterate over each month
    for month, group in df.groupby('Month'):
        group = group.copy()
        group['cluster'] = (group['Start_Cluster_Std']).cumsum()
        clusters = group[group['Exceeds_Std'] == 1]
        
        if not clusters.empty:
            for cluster_id, cluster in clusters.groupby('cluster'):
                duration = len(cluster)
                signal_value = signal_dict.get(month, np.nan)
                max_intensity = cluster[var_interest].max()
                
                result.append({
                    'Month': month,
                    'Signal': signal_value,
                    'Frequency': 1,
                    'Cluster_ID': f"{month}.{cluster_id}",
                    'Duration': duration,
                    'Intensity': max_intensity
                })
        else:
            # Handle months with no exceedances
            signal_value = signal_dict.get(month, np.nan)
            max_intensity = group[var_interest].max()

            result.append({
                'Month': month,
                'Signal': signal_value,
                'Frequency': 0,
                'Cluster_ID': f"{month}.0",
                'Duration': 0,
                'Intensity': max_intensity
            })

    # Convert to DataFrame
    result_df = pd.DataFrame(result)

    # Aggregate frequency per month
    frequency_df = result_df.groupby('Month')['Frequency'].sum().reset_index()
    result_df = result_df.drop('Frequency', axis=1).merge(frequency_df, on='Month', how='left')

    # Calculate monthly statistics
    monthly_df = result_df.groupby('Month').agg(
        mean_duration=('Duration', 'mean'),
        mean_frequency=('Frequency', 'mean'),
        mean_intensity=('Intensity', 'mean'),
        signal=('Signal', 'mean')
    ).reset_index()

    return result_df, monthly_df

In [4]:
# Function to get the first and last non-NA dates for each column
def get_valid_date_range(df):
    first_valid_dates = df.apply(lambda col: col.first_valid_index())
    last_valid_dates = df.apply(lambda col: col.last_valid_index())
    return first_valid_dates, last_valid_dates

In [5]:
# Encoder + Temporary Forecasting Head
class TransformerMTS_Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=2, nhead=4):
        super(TransformerMTS_Encoder, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.pos_encoder = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=nhead, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.pos_encoder, num_layers=num_layers)

    def forward(self, x):
        x = self.input_layer(x)  # Shape: [batch, seq_len, hidden_dim]
        x = self.transformer_encoder(x)  # Shape: [batch, seq_len, hidden_dim]
        embedding = x[:, -1, :]  # Final hidden state
        return embedding

class EncoderWithForecastHead(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, pred_horizon, num_layers=2, nhead=4):
        super(EncoderWithForecastHead, self).__init__()
        self.encoder = TransformerMTS_Encoder(input_dim, hidden_dim, num_layers, nhead)
        self.forecast_head = nn.Linear(hidden_dim, output_dim * pred_horizon)
        self.pred_horizon = pred_horizon
        self.output_dim = output_dim

    def forward(self, x):
        embedding = self.encoder(x)
        out = self.forecast_head(embedding)
        out = out.view(-1, self.pred_horizon, self.output_dim)
        return out

In [6]:
# Function to create input-output sequences for multi-step forecasting
def create_sequences(data, target_columns, seq_length=120, pred_horizon=12):
    sequences, targets = [], []
    target_data = data[target_columns]

    for i in range(len(data) - seq_length - pred_horizon + 1):
        seq = data.iloc[i:i+seq_length].values  # Past `seq_length` months
        target = target_data.iloc[i+seq_length:i+seq_length+pred_horizon].values  # Next `pred_horizon` months
        sequences.append(seq)
        targets.append(target)

    return torch.tensor(sequences, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32)

In [7]:
def knn_retrieve_and_blend(query_window, datastore, encoder, base_forecast, K=5, temperature=1.0, alpha=1.0):
    encoder.eval()
    with torch.no_grad():
        query = encoder(query_window).squeeze()

    # Compute distances to all points in the datastore
    distances = torch.stack([torch.norm(query - key) for key, _ in datastore])
    topk_indices = torch.topk(-distances, K).indices

    neighbors = [datastore[i] for i in topk_indices]
    weights = torch.softmax(-distances[topk_indices] / temperature, dim=0)
    neighbor_targets = torch.stack([y for _, y in neighbors])  # Shape: [K, pred_horizon, num_nodes]

    # Compute weighted kNN forecast
    knn_forecast = torch.sum(weights[:, None, None] * neighbor_targets, dim=0)

    # Distance-adaptive lambda blending weight
    avg_distance = distances[topk_indices].mean().item()
    lam = alpha / (avg_distance + alpha)

    # Final blended forecast
    blended_forecast = (1 - lam) * base_forecast + lam * knn_forecast
    return blended_forecast

In [8]:
def plot_annual_max_series(time, time_series, data_type, data_unit, f_100, ann_percentile_95, percentile_50):
    """
    Plots the annual maximum time series and highlights specific statistical thresholds.

    Parameters:
    - time: The time or year associated with each data point.
    - time_series: The values of the time series to plot.
    - data_type: The type of data being plotted (e.g., 'Rainfall').
    - data_unit: The unit of measurement for the data (e.g., 'mm').
    - f_100: The 100-year flood threshold value.
    - ann_percentile_95: The 95th percentile value.
    - percentile_50: The 50th percentile daily flow value.
    """
    plt.figure(figsize=(10, 6))
    plt.plot(time, time_series, marker='o', linestyle='-', color='b')
    plt.title(f'Annual Maximum {data_type} Over Years')
    plt.xlabel('Time')
    plt.ylabel(f'{data_type} {data_unit}')
    
    # Highlight statistical thresholds
    plt.axhline(y=f_100, color='r', linestyle='--', label=f"100 Year Flood: {f_100:.0f}")
    plt.axhline(y=ann_percentile_95, color='g', linestyle='--', label=f"95th Percentile: {ann_percentile_95:.0f}")
    plt.axhline(y=percentile_50, color='black', linestyle='--', label=f"50th Daily Percentile: {percentile_50:.0f}")
    
    # Text annotations
    plt.text(min(time), f_100, f"100 Year Flood: {f_100:.0f}", horizontalalignment='left', verticalalignment='bottom', color='r')
    plt.text(min(time), ann_percentile_95, f"20 Year Flood: {ann_percentile_95:.0f}", horizontalalignment='left', verticalalignment='bottom', color='g')
    plt.text(min(time), percentile_50, f"50th Percentile Daily Flow: {percentile_50:.0f}", horizontalalignment='left', verticalalignment='bottom', color='black')
    
    plt.legend()
    plt.show()

In [9]:
def plot_daily_values(df, var_interest, ann_step, base_signal, std_thres, f_100, percentile_50, data_unit):
    """
    Plots daily values against an annual extracted wavelet signal with statistical thresholds.

    Parameters:
    - df: DataFrame containing the daily data.
    - var_interest: String, the column name of interest in 'df'.
    - ann_step: DataFrame containing the annual step function data.
    - base_signal: Float, the baseline signal value for 'Ann_Signal'.
    - std_thres: String or float, standard threshold label for the baseline signal.
    - f_100: Float, the 100-year flood threshold value.
    - percentile_50: Float, the 50th percentile daily flow value.
    - data_unit: String, the unit of measurement for the data (e.g., 'm^3/s').
    """
    plt.figure(figsize=(10, 6))

    if std_thres != 'mean':
        std_thres = 'median' 
    
    # Plot the daily data
    plt.plot(df.index, df[var_interest], label='Daily Values', alpha=0.5)  # Adjusted for visibility
    
    # Step function for extracted signal
    plt.step(ann_step.index, ann_step['Ann_Signal'], label='Annual Signal (Step Function)', where='post')
    
    # Add horizontal lines for median, f_100, and 50th percentile values
    plt.axhline(y=base_signal, color='g', linestyle='--', label=f"Standard Annual Signal ({std_thres}): {base_signal:.0f} {data_unit}")
    plt.axhline(y=f_100, color='r', linestyle='--', label=f"100 Year Flood: {f_100:.0f} {data_unit}")
    plt.axhline(y=percentile_50, color='black', linestyle='--', label=f"50th Daily Percentile: {percentile_50:.0f} {data_unit}")
    
    plt.xlabel('Date')
    plt.ylabel(f'Discharge ({data_unit})')
    plt.title('Daily Values vs Annual Extracted Wavelet Signal')
    plt.legend()
    plt.show()

In [10]:
def plot_violin(data, group_by_columns, y_column, title, y_label, percentile_95=None, f_100=None):
    """
    Creates a violin plot for specified data aggregated by given columns.

    :param data: DataFrame containing the data to plot.
    :param group_by_columns: List of column names to group by.
    :param y_column: The name of the column to be plotted on the y-axis.
    :param title: The title of the plot.
    :param y_label: The label for the y-axis.
    :param percentile_95: Optional; the y-value at which to draw a horizontal line for the 95th percentile.
    :param f_100: Optional; the y-value at which to draw a horizontal line for the 100-year flood.

    """
    # Group the data and reset the index
    grouped_data = data.groupby(group_by_columns)[y_column].mean().reset_index()

    warnings.simplefilter(action='ignore', category=FutureWarning)
    plt.figure(figsize=(10, 6))

    # Create the violin plot
    ax = sns.violinplot(x=group_by_columns[0], y=y_column, data=grouped_data)

    plt.title(title)
    plt.xlabel(group_by_columns[0])
    plt.ylabel(y_label)

    # Optionally add threshold lines
    if percentile_95 is not None:
        plt.axhline(y=percentile_95, color='r', linestyle='--', label='95% Daily Percentile')
    if f_100 is not None:
        plt.axhline(y=f_100, color='b', linestyle='--', label='Empirical CDF 100 Year Flood')

    plt.xticks(rotation=45)

    # Annotation adjustments
    text_y_position = ax.get_ylim()[0] + (ax.get_ylim()[1] - ax.get_ylim()[0]) * 0.05
    counts = grouped_data.groupby(group_by_columns[0]).size().reset_index(name='counts')
    for i, row in counts.iterrows():
        ax.text(i, text_y_position, str(int(row['counts'])), horizontalalignment='center', size='small', color='red', weight='semibold')

    plt.tight_layout()
    if percentile_95 is not None or f_100 is not None:
        plt.legend()

    plt.show()

In [11]:
def plot_storm_intensities(series, base_signal, f_10=0, percentile_95=0, f_100=0):
    """
    Plots storm intensities over the years as a violin plot and the progression of storm intensity by day.

    Parameters:
    - series: DataFrame containing storm data, including 'month', 'intensity', 'index_storms', and 'storm_day'.
    - f_10: The empirical CDF 10 year flood intensity value for reference.
    - percentile_95: The 95th percentile intensity value used for y-axis limit calculation.
    - f_100: The empirical CDF 100 year flood intensity value for reference in the plots.
    """
    warnings.simplefilter(action='ignore', category=FutureWarning)

    # Calculate desired y-axis limits
    y_min = 0
    y_max = max(series['Intensity'].max()*1.1, percentile_95, f_100)

    # Setting the overall figure size
    plt.figure(figsize=(20, 8))

    # ----- Plot 1: Violin Plot -----
    ax1 = plt.subplot(1, 2, 1)  # Create subplot 1
    series['month_str'] = pd.to_datetime(series['month']).dt.to_period('M').astype(str)
    series['month_str'] = pd.Categorical(series['month_str'], ordered=True)
    sns.violinplot(x='month_str', y='daily_flow', data=series, cut=0)

    plt.title('Storm Intensities Over Years')
    plt.xlabel('Time')
    plt.ylabel('Intensity')
    plt.axhline(y=f_10, color='black', linestyle='--', label='10-Year Event')
    plt.axhline(y=base_signal, color='green', linestyle='--', label='Signal Threshold')
    plt.axhline(y=f_100, color='r', linestyle='--', label='Empirical CDF 100 Year Flood')
    plt.xticks(rotation=45)
    plt.legend()

    # Set the same y-axis limits
    ax1.set_ylim([y_min, y_max])

    # ----- Plot 2: Line Plot -----
    ax2 = plt.subplot(1, 2, 2)  # Create subplot 2
    grouped = series.groupby(['month', 'storm_index'])
    unique_months = series['month'].unique()
    colors = plt.cm.coolwarm(np.linspace(0, 1, len(unique_months)))  # Adjust colormap as needed
    year_color_map = dict(zip(unique_months, colors))

    for (month, index_storms), group in grouped:
        plt.plot(group['storm_day'], group['daily_flow'], color=year_color_map[month])

    plt.axhline(y=f_10, color='black', linestyle='--')
    plt.axhline(y=base_signal, color='green', linestyle='--')
    plt.axhline(y=f_100, color='r', linestyle='--')
    plt.title('Storm Intensity Progression by Day')
    plt.xlabel('Storm Day')
    plt.ylabel('Intensity')

    custom_lines = [Line2D([0], [0], color=year_color_map[month], lw=4) for month in unique_months]
    plt.legend(custom_lines, [f'Date {month}' for month in unique_months], loc='best', fontsize='small', title="Storm Years")

    # Set the same y-axis limits
    ax2.set_ylim([y_min, y_max])

    plt.tight_layout()
    plt.show()

In [12]:
def KNN(predict, summary):
    """
    Performs a K-Nearest Neighbors (KNN) resampling based on the signal intensity.
    
    Args:
        predict (pd.DataFrame): A DataFrame containing the 'wave' feature for which we want to find nearest neighbors.
        summary (pd.DataFrame): A DataFrame containing historical data with 'Intensity', 'Duration', and 'Frequency' features.

    Returns:
        pd.DataFrame: A DataFrame of resampled rows from the summary DataFrame with added 'month' and 'sim' columns from the predict DataFrame.
    """
    
    # Determine the number of predictions to process
    N = predict.shape[0]
    
    # Calculate the number of neighbors, k, as the square root of number of observations
    k = int(np.sqrt(summary.shape[0]))
    
    # Initialize a list to collect resampled DataFrame rows
    resampled_rows = []

    # Iterate over each prediction
    for i in range(N):
        # Extract the 'wave' value for the current prediction
        signal_value = predict.iloc[i]["signal"]
        
        # Find the indices of the k nearest neighbors in 'summary' based on the absolute difference in 'Intensity'
        kNN_ind = np.argsort(np.abs(signal_value - summary["signal"]))[:k]

        # Calculate weights for each of the k neighbors inversely proportional to their rank (1 being closest)
        W = [(1/x) / np.sum(1 / np.arange(1, k+1)) for x in np.arange(1, k+1)]
        
        # Ensure the weights sum to 1.0
        assert np.isclose(np.sum(W), 1.0), 'weights should sum to 1'

        # Calculate the cumulative sum of weights to facilitate weighted random sampling
        cumW = np.cumsum(W)
        
        # Generate a random number and use it to select a neighbor based on the weighted distribution
        rnd = np.random.rand()
        sampled_index = next(x for x, val in enumerate(cumW) if val > rnd)
        
        # Get the index of the selected sample
        samp_ind = kNN_ind.iloc[sampled_index]

        # Extract the 'time' and 'sim' values from the current prediction
        month = predict.iloc[i]["month"]
        sim = predict.iloc[i]["sim"]

        # Create a new row from the selected sample with additional 'month', 'sim', and original 'wave' values
        resampled_row = summary.iloc[[samp_ind]][["signal", "mean_intensity", "mean_duration", "mean_frequency"]].copy()
        resampled_row.rename(columns={'signal': 'Scale_Sig', 'mean_intensity': 'Scale_Int', 'mean_duration': 'Scale_Dur', 'mean_frequency': 'Scale_Freq'}, inplace=True)
        resampled_row["month"] = month
        resampled_row["sim"] = sim
        resampled_row["signal"] = signal_value
        
        # Append the new row to the list of resampled rows
        resampled_rows.append(resampled_row)

    # Concatenate all resampled rows into a single DataFrame
    resampled_summary = pd.concat(resampled_rows, ignore_index=True)
    
    # Return the resampled DataFrame
    return resampled_summary

In [13]:
def KNN_MLE(predict, summary):
    """
    Performs a K-Nearest Neighbors (KNN) analysis using maximum likelihood estimates
    based on the signal intensity, duration, and frequency.

    Args:
        predict (pd.DataFrame): A DataFrame containing the 'wave' feature for which we want to find nearest neighbors.
        summary (pd.DataFrame): A DataFrame containing historical data with 'Intensity', 'Duration', and 'Frequency' features.

    Returns:
        pd.DataFrame: A DataFrame with estimated values for 'Intensity', 'Duration', and 'Frequency' for each prediction,
                      along with the 'month' and 'sim' columns from the predict DataFrame.
    """
    
    N = predict.shape[0]
    k = int(np.sqrt(summary.shape[0]))
    resampled_rows = []

    for i in range(N):
        signal_value = predict.iloc[i]["signal"]
        kNN_ind = np.argsort(np.abs(signal_value - summary["signal"]))[:k]
        W = [(1/x) / np.sum(1 / np.arange(1, k+1)) for x in np.arange(1, k+1)]
        
        assert np.isclose(np.sum(W), 1.0), 'weights should sum to 1'

        # Calculate weighted averages for 'Intensity', 'Duration', 'Frequency' instead of sampling
        signal_estimate = np.dot(W, summary.iloc[kNN_ind]["signal"])
        intensity_estimate = np.dot(W, summary.iloc[kNN_ind]["mean_intensity"])
        duration_estimate = np.dot(W, summary.iloc[kNN_ind]["mean_duration"])
        frequency_estimate = np.dot(W, summary.iloc[kNN_ind]["mean_frequency"])
        
        month = predict.iloc[i]["month"]
        sim = predict.iloc[i]["sim"]

        # Create a new row with the estimated values and original 'month', 'sim', and 'wave' values
        estimated_row = pd.DataFrame({
            "month": [month],
            "sim": [sim],
            "signal": [signal_value],
            "Scale_Sig": [signal_estimate],
            "Scale_Int": [intensity_estimate],
            "Scale_Dur": [duration_estimate],
            "Scale_Freq": [frequency_estimate]
        })
        
        resampled_rows.append(estimated_row)

    resampled_summary = pd.concat(resampled_rows, ignore_index=True)
    
    return resampled_summary

In [14]:
def simulate_storm_frequencies(sim, time, future_signal, result_df, params, KNN_sampling = True, plot_pair=False, plot_RV=False):
    """
    Simulates storm statistics for a given simulation ID and forecast data.

    Args:
        sim (int): The simulation ID.
        time (np.array): Array of time for which statistics are to be simulated.
        reconstruct_forecast (pd.DataFrame): DataFrame containing the forecast data.
        result_df (pd.DataFrame): DataFrame used for KNN bootstrap sampling.
        plot_pair (bool): Flag to determine whether to plot pairplots.
        plot_RV (bool): Flag to determine whether to plot univariate pdfs.

    Returns:
        pd.DataFrame: DataFrame containing the simulation results with storm statistics.
        pd.DataFrame: DataFrame containing the chosen distributions for each variable.
    """
    
    # Create parent distribution
    parent = pd.DataFrame({
        'sim': sim,
        'month': time,
        'signal': future_signal['signal'] #['Forecast']  # Assign signal directly
    })

    parent['Intensity'] = 0
    parent['Duration'] = 0

    n_samples = len(result_df['Intensity'].values)
    
    frequency_params = params['frequency'][0]
    signal_params = params['signal_freq'][0]
    freq_dist = params['frequency'][1]
    signal_dist = params['signal_freq'][1]
        
    if signal_dist == 'Expon':
        pass
    elif signal_dist == 'GPD':
        s_gpd_c = signal_params['gpd_c']
        s_gpd_loc = signal_params['gpd_loc']
    elif signal_dist == 'Gamma':
        s_gamma_alpha = signal_params['gamma_alpha']
        s_gamma_loc = signal_params['gamma_loc']
    else:
        _, signal_samples, _ = fit_logspline_density(result_df['Signal'], n_samples, model=signal_params, plot=False) 

    if freq_dist != 'Poisson':
        _, frequency_samples, _ = fit_logspline_density(result_df['Frequency'], n_samples, model=frequency_params, plot=False) 
    
    for index, row in parent.iterrows():
        curr_signal = parent.loc[index, 'signal']
        
        if freq_dist == 'Poisson':
            freq_scale = np.maximum(0, future_signal.loc[index, 'Scale_Freq'])
            frequency_samples = np.random.poisson(freq_scale, n_samples)
        
        if signal_dist == 'GPD':
            sig_scale = np.maximum(0, future_signal.loc[index, 'Scale_Sig'])
            signal_samples = genpareto.rvs(scale=sig_scale, c=s_gpd_c, loc=s_gpd_loc, size=n_samples)
        if signal_dist == 'Expon':
            sig_scale = np.maximum(0, future_signal.loc[index, 'Scale_Sig'])
            signal_samples = np.random.exponential(scale=sig_scale, size=n_samples)
        if signal_dist == 'Gamma':
            sig_scale = np.maximum(0, future_signal.loc[index, 'Scale_Sig'])
            signal_samples = gamma.rvs(scale=sig_scale, a=s_gamma_alpha, loc=s_gamma_loc, size=n_samples)
        
        freq_sample = {
            'Signal': signal_samples,
            'Frequency': np.maximum(0, frequency_samples)
        }
        
        freq_copula = fit_empirical_copula(result_df, freq_sample)

        if KNN_sampling:
            freq_copula = KNN_bootstrap(curr_signal, freq_copula, sample_size=n_samples)
        
        freq_sample = freq_copula.sample(1).iloc[0]
        
        parent.loc[index, 'Frequency'] = freq_sample['Frequency']
        parent.loc[index, 'unique_storm_id'] = f"{sim}_{row['month']}_0"

    return parent

In [15]:
def expand_rows_based_on_frequency(parent, sim):
    """
    Expands each row of the DataFrame based on the 'Frequency' column and assigns unique storm IDs.

    Args:
        parent (pd.DataFrame): DataFrame containing storm data with a 'Frequency' column.
        sim (int): Simulation identifier used for setting unique storm IDs.

    Returns:
        pd.DataFrame: Expanded DataFrame with unique storm IDs that reset for each month.
    """
    # Ensure the DataFrame is sorted by month for consistent processing
    parent = parent.sort_values(by='month')

    # Repeat each row according to the 'Frequency'
    child = parent.loc[parent.index.repeat(parent['Frequency'])].reset_index(drop=True)

    # Generate a unique storm ID for each row, resetting count each month
    child['storm_index'] = child.groupby('month').cumcount() + 1  # Starts counting from 1

    # Create unique identifiers
    child['unique_storm_id'] = child.apply(
        lambda x: f"{sim}_{x['month']}_{x['storm_index']}" if x['Frequency'] > 0 else f"{sim}_{x['month']}_0", axis=1
    )

    # Retain original rows for years with 0 frequency but adjust their IDs
    zero_freq_rows = parent[parent['Frequency'] == 0]
    zero_freq_rows['storm_index'] = 0  # Set storm index to 0 for years with no storms
    zero_freq_rows['unique_storm_id'] = zero_freq_rows.apply(lambda x: f"{sim}_{x['month']}_0", axis=1)

    # Combine the rows with zero frequency with the expanded rows
    final_df = pd.concat([child, zero_freq_rows], ignore_index=True).sort_values(by='month')

    return final_df

In [16]:
def simulate_storm_statistics(sim, parent, result_df, future_signal, params, KNN_sampling = True, plot_pair=False, plot_RV=False):
    """
    Simulates storm statistics for a given simulation ID and forecast data.

    Args:
        sim (int): The simulation ID.
        parent (pd.DataFrame): DataFrame to merge to
        reconstruct_forecast (pd.DataFrame): DataFrame containing the forecast data.
        result_df (pd.DataFrame): DataFrame used for KNN bootstrap sampling.
        plot_pair (bool): Flag to determine whether to plot pairplots.
        plot_RV (bool): Flag to determine whether to plot univariate pdfs.

    Returns:
        pd.DataFrame: DataFrame containing the simulation results with storm statistics.
        pd.DataFrame: DataFrame containing the chosen distributions for each variable.
    """    

    # Initialize Dataframe to update for frequencies greater than 0    
    filtered_parent = parent[(parent['storm_index'] > 0)]

    n_samples = len(result_df['Intensity'].values)
    
    duration_params = params['duration'][0]
    intensity_params = params['intensity'][0]
    signal_params = params['signal'][0]
    intensity_dist = params['intensity'][1]
    dur_dist = params['duration'][1]
    signal_dist = params['signal'][1]
    
    if dur_dist == 'Gamma':
        dur_gamma_alpha = duration_params['gamma_alpha']
        dur_gamma_loc = duration_params['gamma_loc']
    elif dur_dist == 'GPD':
        dur_gpd_c = duration_params['gpd_c']
        dur_gpd_loc = duration_params['gpd_loc']
    elif dur_dist == 'Expon':
        pass
    else:
        _, duration_samples, _ = fit_logspline_density(result_df['Duration'], n_samples, model=duration_params, plot=False) 
    

    if intensity_dist == 'Expon':
        pass
    elif intensity_dist == 'GPD':
        gpd_c = intensity_params['gpd_c']
        gpd_loc = intensity_params['gpd_loc']
    elif intensity_dist == 'Gamma':
        gamma_alpha = intensity_params['gamma_alpha']
        gamma_loc = intensity_params['gamma_loc']
    else:
        _, intensity_samples, _ = fit_logspline_density(result_df['Intensity'], n_samples, model=intensity_params, plot=False) 

    if signal_dist == 'Expon':
        pass
    elif signal_dist == 'GPD':
        s_gpd_c = signal_params['gpd_c']
        s_gpd_loc = signal_params['gpd_loc']
    elif signal_dist == 'Gamma':
        s_gamma_alpha = signal_params['gamma_alpha']
        s_gamma_loc = signal_params['gamma_loc']
    else:
        _, signal_samples, _ = fit_logspline_density(result_df['Signal'], n_samples, model=signal_params, plot=False) 

    for index, row in filtered_parent.iterrows():
        curr_signal = parent.loc[index, 'signal']
        month = parent.loc[index, 'month']
        
        if dur_dist == 'GPD':
            dur_scale = np.maximum(0, future_signal['Scale_Dur'][future_signal['month'] == month])
            duration_samples = genpareto.rvs(scale=dur_scale, c=dur_gpd_c, loc=dur_gpd_loc, size=n_samples)
        elif dur_dist == 'Gamma':
            dur_scale = np.maximum(0, future_signal['Scale_Dur'][future_signal['month'] == month])
            duration_samples = gamma.rvs(scale=dur_scale, a=dur_gamma_alpha, loc=dur_gamma_loc, size=n_samples)
        elif dur_dist == 'Expon':
            dur_scale = np.maximum(0, future_signal['Scale_Dur'][future_signal['month'] == month])
            duration_samples = np.random.exponential(scale=dur_scale, size=n_samples)
        
        if intensity_dist == 'GPD':
            int_scale = np.maximum(0, future_signal['Scale_Int'][future_signal['month'] == month])
            intensity_samples = genpareto.rvs(scale=int_scale, c=gpd_c, loc=gpd_loc, size=n_samples)
        elif intensity_dist == 'Expon':
            int_scale = np.maximum(0, future_signal['Scale_Int'][future_signal['month'] == month])
            intensity_samples = np.random.exponential(scale=int_scale, size=n_samples)
        elif intensity_dist == 'Gamma':
            int_scale = np.maximum(0, future_signal['Scale_Int'][future_signal['month'] == month])
            intensity_samples = gamma.rvs(scale=int_scale, a=gamma_alpha, loc=gamma_loc, size=n_samples)

        if signal_dist == 'GPD':
            sig_scale = np.maximum(0, future_signal['Scale_Sig'][future_signal['month'] == month])
            signal_samples = genpareto.rvs(scale=sig_scale, c=s_gpd_c * signal_scaling, loc=s_gpd_loc, size=n_samples)
        elif signal_dist == 'Expon':
            sig_scale = np.maximum(0, future_signal['Scale_Sig'][future_signal['month'] == month])
            signal_samples = np.random.exponential(scale=sig_scale, size=n_samples)
        elif signal_dist == 'Gamma':
            sig_scale = np.maximum(0, future_signal['Scale_Sig'][future_signal['month'] == month])
            signal_samples = gamma.rvs(scale=sig_scale, a=s_gamma_alpha, loc=s_gamma_loc, size=n_samples)
        
        univariate_sample = {
            'Signal': signal_samples,
            'Intensity': np.maximum(0, intensity_samples),
            'Duration': np.maximum(0, duration_samples),
        }
        
        emp_copula = fit_empirical_copula(result_df, univariate_sample)
    
        if plot_pair:
            FIDS_pairplot(result_df, columns=univariate_sample.keys())
            FIDS_pairplot(emp_copula, columns=univariate_sample.keys())

        if KNN_sampling:
            emp_copula = KNN_bootstrap(curr_signal, emp_copula, sample_size=n_samples)
            
        sample = emp_copula.sample(1).iloc[0]
        parent.loc[index, ['Intensity', 'Duration']] = sample[['Intensity', 'Duration']]

    return parent

In [17]:
def empirical_return_levels(df, cols, targets=(10, 100), resample_rule='A'):
    """
    Compute empirical return‐period levels (e.g. 10-yr, 100-yr) from monthly data.

    Parameters
    ----------
    df : pd.DataFrame
        Monthly data with a DatetimeIndex.
    cols : list[str]
        Column names to process.
    targets : tuple[float]
        Return periods (years) to extract.  Default (10, 100).
    resample_rule : str
        Pandas offset alias for annual aggregation.
        'A' = calendar year max;  e.g. 'A-SEP' for Oct–Sep water year.

    Returns
    -------
    pd.DataFrame
        Index = column names, columns = target return periods.
        Example columns: ['T10', 'T100'] for default targets.
    """
    results = {}
    t_arr = np.asarray(targets)

    for c in cols:
        # annual-maximum series
        ams = (
            df[c]
            .resample(resample_rule)
            .max()
            .dropna()
        )
        n = len(ams)
        if n < 2:
            raise ValueError(f"Series '{c}' has fewer than 2 annual maxima.")

        # Weibull plotting positions
        ams_sorted = ams.sort_values(ascending=False).reset_index(drop=True)
        rank = np.arange(1, n + 1)               # 1 … n
        exceed_prob = rank / (n + 1)             # P(X ≥ x)
        return_period = 1 / exceed_prob          # T (years)

        # Linear interpolation in log-space for stability (optional)
        rl = np.interp(t_arr, return_period[::-1], ams_sorted[::-1])

        results[c] = rl

    col_names = [f"T{int(t)}" for t in targets]
    return pd.DataFrame(results, index=col_names).T

In [18]:
def generate_forecasts(df, model, datastore, target_cities, seq_length, pred_horizon):
    """
    Returns a dictionary of actual and predicted values for each city over the last forecast window.
    """
    latest_qstart = len(df) - (seq_length + pred_horizon)
    query_window = torch.tensor(
        df.iloc[latest_qstart : latest_qstart + seq_length].values,
        dtype=torch.float32
    ).unsqueeze(0)

    base_forecast = model(query_window).squeeze()
    blended_np    = knn_retrieve_and_blend(query_window, datastore, model.encoder, base_forecast).detach().numpy()

    actual_future = df[target_cities].iloc[
        latest_qstart + seq_length : latest_qstart + seq_length + pred_horizon
    ].values

    forecast_index = pd.date_range(
        start=df.index[latest_qstart + seq_length],
        periods=pred_horizon,
        freq='MS'
    )

    return {
        "forecast_index": forecast_index,
        "actual": actual_future,
        "predicted": blended_np,
        "target_cities": target_cities
    }

In [19]:
def plot_forecast_results(results_dict):
    """
    Plots forecast vs actual for each city using multivariate results.
    """
    actual     = results_dict["actual"]
    predicted  = results_dict["predicted"]
    index      = results_dict["forecast_index"]
    cities     = results_dict["target_cities"]

    n = len(cities)
    fig, axs = plt.subplots(2, 2, figsize=(14, 10), sharex=True)

    for i, city in enumerate(cities):
        ax = axs[i // 2, i % 2]

        ax.plot(index, actual[:, i],   "--", color="blue", label="Actual")
        ax.plot(index, predicted[:, i], "-", color="red",  label="Predicted")

        mse = mean_squared_error(actual[:, i], predicted[:, i])
        mae = np.abs(actual[:, i] - predicted[:, i]).mean()

        ax.set_title(f"{city}\nMSE: {mse:.3f} | MAE: {mae:.3f}")
        ax.set_ylabel("Value")
        ax.grid(True)
        ax.legend()

    axs[-1, 0].set_xlabel("Time")
    axs[-1, 1].set_xlabel("Time")
    plt.suptitle("Transformer Encoder (Blended kNN-MTS) | Last-Step Forecast vs Actual", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

In [20]:
def single_split(df: pd.DataFrame, label: str) -> tuple[pd.Index, pd.Index]:
    """
    Return (train_index, test_index) for one of:
        std1, std2, std3, kblock1-3, or ksplit1-3
    """
    yr = 12  # months per year
    n = len(df)  # expected to be ~92 * 12 = 1104

    lb = label.lower().strip()

    if lb in {"std1", "std2", "std3"}:
        # Define the (train_years, test_years) for each std version
        std_configs = {
            "std1": (50, 15),
            "std2": (65, 15),
            "std3": (70, 20),
        }
        train_yrs, test_yrs = std_configs[lb]
        train_end = train_yrs * yr
        test_end = train_end + test_yrs * yr
        if test_end > n:
            raise ValueError(f"{lb} split exceeds data length")
        return df.index[:train_end], df.index[train_end:test_end]

    elif lb == "std":
        # Original std = std2 for backward compatibility
        train_end = 75 * yr
        test_end  = train_end + 15 * yr
        if test_end > n:
            raise ValueError("std split exceeds data length")
        return df.index[:train_end], df.index[train_end:test_end]

    elif lb.startswith("kblock"):
        which = int(lb[-1])
        train_years = [25, 50, 75][which - 1]
        test_years = 10
        train_end = train_years * yr
        test_end = train_end + test_years * yr
        if test_end > n:
            raise ValueError("kblock split exceeds data length")
        return df.index[:train_end], df.index[train_end:test_end]

    elif lb.startswith("ksplit"):
        which = int(lb[-1])
        chunk_start = (which - 1) * 30 * yr
        train_years, test_years = 25, 5
        train_start = chunk_start
        train_end = train_start + train_years * yr
        test_start = train_end
        test_end = test_start + test_years * yr
        if test_end > n:
            raise ValueError("ksplit split exceeds data length")
        return df.index[train_start:train_end], df.index[test_start:test_end]

    else:
        raise ValueError("label must be one of: std, std1-3, kblock1-3, ksplit1-3")

## Parameter Inputs

In [21]:
# Input file information
var_interest = 'discharge_cfs'  # Variable of interest name in source file for time series data

# Naming and formatting
data_type = "Streamflow" # Variable name in time series
data_unit = "cfs" # Variable unit in time series
run_name = "Control" # Name your file output
folder = 'Test'  # Specify desired output folder
sns.set_theme(style = 'white') # Set theme for all plottings
validation_sim = True # Forecasting or validating
validate_type = "ksplit2" # std, kblock (1,2,3), ksplit (1,2,3)
full_pred = True

# Parameters
# GENERAL
steps = 60  # time length for forecasting, default 5 years ahead (60)
n_simulations = 1000 # default 1000

# OSCILLATIONS
sigtest = 'default'   # Type of wavelet sigtest 'red' or 'white', default will plot both but use red for extraction
siglvl = 0.95  # choice of significance level for wavelet extraction, default is 0.95.

# FORECAST
seq_length = 60   # Use 7 to 10 years of past data
pred_horizon = steps
hidden_dim = 64
num_layers = 2   # Transformer Encoder Layers
lr = 0.001       # Learning rate
epochs = 100
batch_size = 16
early_stop = False

# CLUSTERING
std_thres = 'default'  # choice of threshold for standard exceedances ("median", "mean"), default of median.
nonstationary_type = 'KNN' # Define nonstationary type ('KNN', 'KNN_MLE', 'scaled','stationary'). Default KNN.
KNN_sampling = True # Decide whether to sample from KNN for final multivariate storm distribution. Default True.
signal_dist = 'Expon' # Set dist to extract intensities ('Expon', 'Gamma' or 'GPD'). Default is Gamma.
intensity_dist = 'Expon' # Set dist to extract intensities ('Expon', 'Gamma' or 'GPD'). Default is Gamma.
duration_dist = 'Expon' # Set dist to extract durations ('Expon', 'Gamma', or 'GPD'). Default is Exponential.
frequency_dist = 'Poisson' # Set dist to extract frequencies ('Poisson'). Default is Poisson.
dist_fix = True # Decide whether to fix distribution fits. If True dists are fixed to those specified, otherwise defaults on fit based on KS/Chi Squared Test.

# EXPORT OPTIONS (Note that the code will always export a csv of all simulated storms)
save_signal = False  # Save the wavelet reconstructed signal in a separate csv with residuals from historical timeseries
save_forecast_signal = False  # Save the forecasted reconstructed signal in a separate csv
save_summ_exceeds = True  # Save summary data on threshold exceedances (needed for validation plots across entire period of interest)
record_dists = False  # Save the types of distributions fit during simulation
save_params = True # Save the nonstationary parameters used

# PLOT OPTIONS
plot_raw = False
plot_LASSO = False
plot_transform = False
plot_wave = False
plot_reconstruct = False
plot_innovations = False
plot_BAIC = False
plot_wave_forecasts = False
plot_forecasts = False
plot_traj = False
plot_RV = False
plot_pair = False
plot_allsims = False
plot_onesim = False

# Multivariate Data Load

In [22]:
climate_ind = pd.read_csv('Climatological/climate_indices_no_pna.csv', parse_dates=[0], index_col=0)

In [23]:
merged_discharge = pd.read_csv("Discharge_Preprocess.csv", parse_dates=["datetime"], index_col="datetime")
merged_discharge = merged_discharge.apply(pd.to_numeric, errors='coerce')

In [24]:
# Ensure both indices have the same timezone
if climate_ind.index.tz is not None:
    climate_ind = climate_ind.tz_convert(None)
if merged_discharge.index.tz is not None:
    merged_discharge = merged_discharge.tz_convert(None)

# Get first and last non-NA dates for both datasets
first_valid_climate, last_valid_climate = get_valid_date_range(climate_ind)
first_valid_discharge, last_valid_discharge = get_valid_date_range(merged_discharge)

# Determine the latest of the first non-NA dates (max of min dates)
start_date = max(first_valid_climate.max(), first_valid_discharge.max())

# Determine the earliest of the last non-NA dates (min of max dates)
end_date = min(last_valid_climate.min(), last_valid_discharge.min())

# Clip the data to the valid date range
climate_filtered = climate_ind.loc[start_date:end_date]
discharge_filtered = merged_discharge.loc[start_date:end_date]

# Merge the filtered dataframes
merged_data = climate_filtered.merge(discharge_filtered, left_index=True, right_index=True, how='inner')

# Resample data to monthly frequency and compute the maximum for each month
all_data_mon_max = merged_data.resample('ME').max()

# Resample discharge data to monthly sums
discharge_mon_sum = merged_data.resample('ME').sum()

# Normalize each column: subtract mean and divide by std
discharge_normalized = (discharge_mon_sum - discharge_mon_sum.mean()) / discharge_mon_sum.std()

# Rename columns to have '_sum' suffix
discharge_normalized.columns = [f"{col}_sum" for col in discharge_normalized.columns]

# Merge monthly sums into the main DataFrame
all_data_mon_max = all_data_mon_max.join(discharge_normalized, how='left')
all_data_mon_max.index = all_data_mon_max.index.to_period('M').to_timestamp()

# Extract month from index
all_data_mon_max['month'] = all_data_mon_max.index.month

# Create cyclic (sin/cos) features
all_data_mon_max['sin_month'] = np.sin(2 * np.pi * all_data_mon_max['month'] / 12)

In [25]:
minn_mean = all_data_mon_max['minneapolis'].mean()
sl_mean = all_data_mon_max['st_louis'].mean()
kc_mean = all_data_mon_max['kansas_city'].mean()
clin_mean = all_data_mon_max['clinton'].mean()
minn_std = all_data_mon_max['minneapolis'].std()
sl_std = all_data_mon_max['st_louis'].std()
kc_std = all_data_mon_max['kansas_city'].std()
clin_std = all_data_mon_max['clinton'].std()

In [26]:
cols_to_process = ['minneapolis', 'clinton', 'kansas_city', 'st_louis']
rl_df = empirical_return_levels(all_data_mon_max, cols_to_process)

print(rl_df)

                       T10       T100
minneapolis   49334.782609    90300.0
clinton      220260.869565   307000.0
kansas_city  258478.260870   558000.0
st_louis     793173.913043  1050000.0


/tmp/ipykernel_925/2195537229.py:30: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  .resample(resample_rule)


In [27]:
if validation_sim:
    train_idx, test_idx = single_split(all_data_mon_max, validate_type)
    train_df = all_data_mon_max.loc[train_idx]
    test_df  = all_data_mon_max.loc[test_idx]
    all_data_mon_max = train_df
    print(validate_type, train_df.shape, test_df.shape)

    # Save test set
    folder = 'CV_Experiments'
    output_dir = f"{folder}/Test_Sets"
    os.makedirs(output_dir, exist_ok=True)  # Create folder if it doesn't exist

    test_file_path = os.path.join(output_dir, f"{validate_type}_TestVals.csv")
    test_df.to_csv(test_file_path, index=True)
    run_name = validate_type
    save_summ_exceeds = False
    
    if full_pred:
        steps = len(test_df)
        pred_horizon = steps

ksplit2 (300, 26) (60, 26)


## Wavelet

In [28]:
sig_scales_all = []

cols = ['minneapolis_norm', 'clinton_norm', 'kansas_city_norm','st_louis_norm', 'minneapolis_norm_sum', 'clinton_norm_sum', 'kansas_city_norm_sum','st_louis_norm_sum']

for column in cols:
    print(f"Processing {column} ...")
    col_copy = all_data_mon_max[column].copy()  # Make a copy
    col_copy.dropna(inplace=True)  # Drop NaN values
    data_source = column
    
    wlt = wavelet(col_copy)
    Cw = CI(wlt, col_copy, siglvl, "r")
    C = CI(wlt, col_copy, siglvl, "w")
    
    year_month = col_copy.index.to_period('M').to_timestamp()
    time = np.unique(year_month)
    
    # Global Wavelet Spectrum
    plt_dataset = {
        'Time': time,
        'Period': wlt['period'],
        'Avg_Power': wlt['avg_power'],
        'Power': wlt['power'],
        'COI': wlt['coi'],
        'W_noise': C['sig'],
        'R_noise': Cw['sig']
    }

    if plot_wave:
        wavelet_plot(plt_dataset, siglvl, data_source, sigtest)
    print("Red Noise Sig Test Results")
    reconstruction, sig_scales = reconstruct(Cw, col_copy)
    sig_scales_all.append(sig_scales)

    # Add reconstructed series to the full monthly dataframe
    wave_col_name = f"{column}_wave"
    reconstructed_series = pd.Series(reconstruction, index=col_copy.index, name=wave_col_name)

    # Merge into main DataFrame
    all_data_mon_max = all_data_mon_max.join(reconstructed_series, how='left')

    # Create lagged wavelet feature
    wave_lag_col_name = f"{column}_wave_lag"
    reconstructed_series_lagged = reconstructed_series.shift(1).rename(wave_lag_col_name)

    # Merge lagged wavelet feature
    all_data_mon_max = all_data_mon_max.join(reconstructed_series_lagged, how='left')

/tmp/ipykernel_925/2158036133.py:39: RuntimeWarning: divide by zero encountered in divide
  expnt = -(scl * k - param) ** 2 / (2 * (k > 0))
/tmp/ipykernel_925/2158036133.py:125: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha = model.params[0] # Equation 15, extract lag-1 autocorr


Processing minneapolis_norm ...
Red Noise AR1 Coefficient: 0.5846517027805967
Red Noise Sig Test Results
Significant scales:
[0.28 0.28 0.29 0.36 0.36 0.37 0.38 0.38 0.39 0.4  0.4  0.41 0.42 0.42
 0.43 0.44 0.45 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61 0.62 0.63
 0.64 0.66 0.67 0.68 0.69 0.7  0.71 0.73 0.74 0.75 0.77 0.78 0.79 0.81
 0.82 0.84 0.85 0.86 0.88 0.9  0.91 1.06 1.08 1.1  1.12 1.14 1.16 1.36
 1.38 1.4  1.43 1.45 1.48 1.51 1.53 1.56 1.59 1.61 1.64 1.67 1.7  1.73
 1.76 1.79 1.82 1.85 1.89 1.92 1.95 1.99 2.02 2.24 2.28 2.32 2.36 2.4
 2.45 2.49 2.53 2.58 2.62 2.67 2.71 2.76 2.81 2.86 2.91 2.96 3.01 3.06
 3.12 3.17 3.23 3.28 3.34 3.4  3.46 3.52 3.58 3.64 3.71 3.77 3.84 3.9
 3.97 4.04 4.11 4.18 4.26 5.33 5.43 5.52 5.62 5.72 5.82 5.92 6.02 6.13
 6.23 6.34 6.45 6.57 6.68]
Processing clinton_norm ...


/tmp/ipykernel_925/2158036133.py:39: RuntimeWarning: divide by zero encountered in divide
  expnt = -(scl * k - param) ** 2 / (2 * (k > 0))


Red Noise AR1 Coefficient: 0.6740458406012292
Red Noise Sig Test Results
Significant scales:
[0.28 0.28 0.36 0.36 0.37 0.38 0.38 0.39 0.4  0.4  0.41 0.42 0.42 0.43
 0.44 0.45 0.46 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61 0.62
 0.63 0.64 0.66 0.67 0.68 0.69 0.7  0.71 0.73 0.74 0.75 0.77 0.78 0.79
 0.81 0.82 0.84 0.85 0.86 0.88 0.9  0.91 1.03 1.05 1.06 1.08 1.1  1.12
 1.14 1.16 1.18 1.2  1.22 1.24 1.38 1.4  1.43 1.45 1.48 1.51 1.53 1.56
 1.59 1.61 1.64 1.67 1.7  1.73 1.76 1.79 1.82 1.85 1.89 1.92 1.95 1.99
 2.02 2.06 2.09 2.13 2.17 2.2  2.24 2.28 2.32 2.36 2.4  2.45 2.49 2.53
 2.58 2.62 2.67 2.71 2.76 2.81 2.86 2.91 2.96 3.01 3.06 3.12 3.17 3.23
 3.28 3.34 3.4  3.46 3.52 3.58 3.64 3.71 3.77 3.84 3.9  3.97 4.04 4.11
 4.18 4.26 4.33 4.41 4.48 4.56 4.64 4.72 4.81 4.89 4.98 5.06 5.15 5.24
 5.33 5.43 5.52 5.62 5.72 5.82 5.92 6.02 6.13 6.23 6.34 6.45 6.57 6.68
 6.8  6.92 7.04 7.16]
Processing kansas_city_norm ...
Red Noise AR1 Coefficient: 0.6908298789359986


/tmp/ipykernel_925/2158036133.py:125: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha = model.params[0] # Equation 15, extract lag-1 autocorr
/tmp/ipykernel_925/2158036133.py:39: RuntimeWarning: divide by zero encountered in divide
  expnt = -(scl * k - param) ** 2 / (2 * (k > 0))
/tmp/ipykernel_925/2158036133.py:125: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha = model.params[0] # Equation 15, extract lag-1 autocorr
/tmp/ipykernel_925/2158036133.py:39: RuntimeWarning: divide by zero encountered in divide
  expnt = -(scl * k - param) ** 2 / (2 * (k > 0))


...

/tmp/ipykernel_925/2158036133.py:125: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha = model.params[0] # Equation 15, extract lag-1 autocorr
/tmp/ipykernel_925/2158036133.py:39: RuntimeWarning: divide by zero encountered in divide
  expnt = -(scl * k - param) ** 2 / (2 * (k > 0))
/tmp/ipykernel_925/2158036133.py:125: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha = model.params[0] # Equation 15, extract lag-1 autocorr


Red Noise Sig Test Results
Significant scales:
[0.36 0.37 0.38 0.38 0.39 0.4  0.4  0.41 0.42 0.42 0.43 0.44 0.45 0.53
 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61 0.62 0.63 0.64 0.66 0.67 0.68
 0.69 0.7  0.71 0.73 0.74 0.75 0.77 0.78 0.79 0.81 0.82 0.84 0.85 0.86
 0.88 0.9  1.1  1.56 1.59 1.61 1.64 1.67 1.7  1.73 1.76 1.79 1.82 1.85
 2.32 2.36 2.4  2.45 2.49 2.53 2.58 2.62 2.67 2.71 2.76 2.81 2.86 2.91
 2.96 3.01 3.06 3.12 3.17 3.23 3.28 3.34 3.4  3.46 3.52 3.58 3.64 3.71
 3.77 3.84 3.9  3.97 4.04 4.11 4.18 4.26 4.33 4.41 4.48 4.56 5.15 5.24
 5.33 5.43 5.52 5.62 5.72 5.82 5.92 6.02 6.13 6.23 6.34 6.45 6.57 6.68
 6.8  6.92 7.04]
Processing clinton_norm_sum ...
Red Noise AR1 Coefficient: 0.5892068963024907
Red Noise Sig Test Results
Significant scales:
[ 0.36  0.37  0.38  0.38  0.39  0.4   0.4   0.41  0.42  0.42  0.43  0.44
  0.45  0.52  0.53  0.54  0.55  0.56  0.57  0.58  0.59  0.6   0.61  0.62
  0.63  0.64  0.66  0.67  0.68  0.69  0.7   0.71  0.73  0.74  0.75  0.77
  0.78  0.79  0.81  0.82

/tmp/ipykernel_925/2158036133.py:39: RuntimeWarning: divide by zero encountered in divide
  expnt = -(scl * k - param) ** 2 / (2 * (k > 0))
/tmp/ipykernel_925/2158036133.py:125: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha = model.params[0] # Equation 15, extract lag-1 autocorr
/tmp/ipykernel_925/2158036133.py:39: RuntimeWarning: divide by zero encountered in divide
  expnt = -(scl * k - param) ** 2 / (2 * (k > 0))
/tmp/ipykernel_925/2158036133.py:125: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha = model.params[0] # Equation 15, extract lag-1 autocorr


Red Noise Sig Test Results
Significant scales:
[ 0.38  0.39  0.4   0.4   0.41  0.42  0.42  0.55  0.56  0.57  0.58  0.59
  0.6   0.61  0.62  0.63  0.64  0.66  0.67  0.68  0.69  0.7   0.71  0.73
  0.74  0.75  0.77  0.78  0.79  0.81  0.82  0.84  0.85  0.86  0.88  1.08
  1.1   1.12  1.14  1.16  1.18  1.2   1.43  1.45  1.48  1.51  1.53  1.56
  1.59  1.61  1.64  1.67  1.7   1.73  1.76  1.79  1.82  1.85  1.89  1.92
  1.95  1.99  2.02  2.06  2.09  2.13  2.17  2.2   2.24  2.28  2.32  2.36
  2.4   2.45  2.49  2.53  2.58  2.62  2.67  2.71  2.76  2.81  2.86  2.91
  2.96  3.01  3.06  3.12  3.17  3.23  3.28  3.34  3.4   3.46  3.52  3.58
  3.64  3.71  3.77  3.84  3.9   3.97  4.04  4.11  4.18  4.26  4.33  4.41
  4.48  4.56  4.64  4.72  4.81  4.89  4.98  5.06  5.15  5.24  5.33  5.43
  5.52  5.62  5.72  5.82  5.92  6.02  6.13  6.23  6.34  6.45 10.67 10.85
 11.04 11.24 11.43 11.63 11.84 12.04 12.25 12.47 12.68 12.91 13.13 13.36
 13.6  13.83 14.07 14.32 14.57 14.83]
Processing st_louis_norm_sum ...
Red No

/tmp/ipykernel_925/2158036133.py:39: RuntimeWarning: divide by zero encountered in divide
  expnt = -(scl * k - param) ** 2 / (2 * (k > 0))
/tmp/ipykernel_925/2158036133.py:125: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha = model.params[0] # Equation 15, extract lag-1 autocorr


## Transformer Forecast

In [29]:
# Load and preprocess data (including all target cities)
df = all_data_mon_max[['nao', 'pdo', 'enso', 'gta',
                       'minneapolis_norm_wave', 'clinton_norm_wave', 'kansas_city_norm_wave', 'st_louis_norm_wave',
                       'minneapolis_norm_sum_wave', 'clinton_norm_sum_wave', 'kansas_city_norm_sum_wave', 'st_louis_norm_sum_wave',
                       'minneapolis_norm_wave_lag', 'clinton_norm_wave_lag', 'kansas_city_norm_wave_lag', 'st_louis_norm_wave_lag','sin_month',
                      ]].dropna()

wavelet_signal = all_data_mon_max[[
                       'minneapolis_norm_wave', 'clinton_norm_wave', 'kansas_city_norm_wave', 'st_louis_norm_wave'
                      ]].dropna()

# Define target variables (cities we want to predict)
target_cities = ['st_louis_norm_wave', 'minneapolis_norm_wave', 'clinton_norm_wave', 'kansas_city_norm_wave']

# 80 % train+val  |  20 % final test
split_90 = int(len(df)*0.9)
train_df   = df.iloc[:split_90]
val_df  = df.iloc[split_90:]

# sequences
X_train, y_train = create_sequences(train_df, target_cities, seq_length, pred_horizon)
X_val,   y_val   = create_sequences(val_df,   target_cities, seq_length, pred_horizon)

/tmp/ipykernel_925/2130423005.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1730833640211/work/torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(sequences, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32)


In [30]:
# ----------------- 2. MODEL SET-UP -----------------
input_dim     = X_train.shape[2]
output_dim    = len(target_cities)
model         = EncoderWithForecastHead(input_dim, hidden_dim, output_dim, pred_horizon)
criterion     = nn.MSELoss()
optimizer     = optim.Adam(model.parameters(), lr=lr)
train_batches = len(X_train) // batch_size
val_batches   = max(1, len(X_val)  // batch_size)

In [31]:
# --------------- 3. TRAINING LOOP ---------------
best_val   = float("inf")
patience   = 10
no_improve = 0

for epoch in range(epochs):
    # ---------- TRAIN ----------
    model.train()
    train_se = 0.0
    n_train  = 0
    for i in range(0, len(X_train), batch_size):
        xb, yb = X_train[i:i+batch_size], y_train[i:i+batch_size]
        optimizer.zero_grad()
        loss = criterion(model(xb), yb)
        loss.backward()
        optimizer.step()
        train_se += loss.item() * xb.size(0)   # accumulate SSE
        n_train  += xb.size(0)

    train_loss = train_se / n_train            # mean MSE on the epoch

    # ---------- VALIDATION ----------
    model.eval()
    with torch.no_grad():
        if len(X_val) == 0:                    # fallback to train metrics
            val_loss = train_loss
        else:
            val_se = 0.0
            n_val  = 0
            for i in range(0, len(X_val), batch_size):
                xb = X_val[i:i+batch_size]
                yb = y_val[i:i+batch_size]
                val_se += criterion(model(xb), yb).item() * xb.size(0)
                n_val  += xb.size(0)
            val_loss = val_se / n_val          # mean MSE

    # ---------- LOG ----------
    if epoch % 10 == 0:
        print(f"Epoch {epoch:3d}/{epochs}"
              f" | Train {train_loss:.6f}"
              f" | Val {val_loss:.6f}")

    # ---------- EARLY-STOP ----------
    if val_loss < best_val - 1e-6:
        best_val   = val_loss
        best_state = model.state_dict()
        no_improve = 0
    else:
        no_improve += 1
        if no_improve >= patience:
            if early_stop:
                print("Early stopping triggered.")
                break

# restore best weights (optional)
model.load_state_dict(best_state)

Epoch   0/100 | Train 2.577225 | Val 2.577225
Epoch  10/100 | Train 1.463257 | Val 1.463257
Epoch  20/100 | Train 0.959753 | Val 0.959753
Epoch  30/100 | Train 0.658637 | Val 0.658637
Epoch  40/100 | Train 0.520467 | Val 0.520467
Epoch  50/100 | Train 0.414212 | Val 0.414212
Epoch  60/100 | Train 0.326762 | Val 0.326762
Epoch  70/100 | Train 0.299384 | Val 0.299384
Epoch  80/100 | Train 0.260550 | Val 0.260550
Epoch  90/100 | Train 0.227202 | Val 0.227202


<All keys matched successfully>

In [32]:
datastore = []

model.eval()
with torch.no_grad():
    for i in range(len(X_train)):
        x_window = X_train[i:i+1]
        embedding = model.encoder(x_window).squeeze()
        future_target = y_train[i]
        datastore.append((embedding.cpu(), future_target.cpu()))

In [33]:
results = generate_forecasts(df, model, datastore, target_cities, seq_length, pred_horizon)
forecast = results['predicted']
if plot_forecasts:
    plot_forecast_results(results)

## Sub-annual clustering parameterization

In [34]:
target_cities = ['minneapolis','clinton','kansas_city','st_louis']
target_cities_norm = ['minneapolis_norm','clinton_norm','kansas_city_norm','st_louis_norm']
df = df[['minneapolis_norm_wave','clinton_norm_wave','kansas_city_norm_wave','st_louis_norm_wave']].rename(columns={
    'minneapolis_norm_wave':'minneapolis',
    'clinton_norm_wave':'clinton',
    'kansas_city_norm_wave':'kansas_city',
    'st_louis_norm_wave':'st_louis'
    })

In [35]:
if std_thres == 'mean':
    thres = df[target_cities].median()
else:
    thres = df[target_cities].mean()

signal_sd = df[target_cities].std()

daily = merged_discharge[target_cities_norm].copy()
daily = daily.rename(columns={
    'minneapolis_norm':'minneapolis',
    'clinton_norm':'clinton',
    'kansas_city_norm':'kansas_city',
    'st_louis_norm':'st_louis'
    })

wavelet_signal = wavelet_signal.rename(columns={
    'minneapolis_norm_wave':'minneapolis',
    'clinton_norm_wave':'clinton',
    'kansas_city_norm_wave':'kansas_city',
    'st_louis_norm_wave':'st_louis'
    })

In [36]:
means = {}
stds = {}
for var in target_cities:
    means[var] = merged_discharge[var].mean()
    stds[var] = merged_discharge[var].std()

In [37]:
# Extract historic flood events
result_dfs = []
multivar_signals = []
cluster_dicts = []

for var in target_cities:
    result_df, multivar_signal = extract_exceedance_clusters(daily, wavelet_signal[[var]], var, float(thres[[var]].iloc[0]))
    
    result_df = result_df.dropna()
    result_df['Intensity'] = (result_df['Intensity'] * stds[var])+means[var]
    result_df['Signal'] = ((result_df['Signal'] * stds[var]) + means[var]).clip(lower=0)
    result_dfs.append(result_df)

    multivar_signal = multivar_signal.dropna()
    multivar_signal['mean_intensity'] = (multivar_signal['mean_intensity'] * stds[var])+means[var]
    multivar_signal['signal'] = ((multivar_signal['signal'] * stds[var]) + means[var]).clip(lower=0)
    multivar_signals.append(multivar_signal)
    
    cluster_dict_std, sigma0_std = trajectory_dict_plot(daily, plot_traj)
    cluster_dicts.append(cluster_dict_std)

    if save_summ_exceeds:
        result_df.to_csv(f"{folder}/Standard_Summary_Exceedances_{run_name}_{var}.csv", index=False)

    if plot_pair:
        # Plot original distribution of storm signal, frequence, intensity, and duration
        FIDS_pairplot(result_df, jitter_col=['Frequency'],columns=['Signal', 'Frequency', 'Intensity','Duration'])

In [38]:
# Fit univariate distributions
if dist_fix == True:
    dist_fix = -1
else:
    dist_fix = 0

dist_parameters = []

for result_df in result_dfs:    
    freq_signal_params, freq_og_signal_samples, freq_signal_dist, freq_sig_test = fit_intensity_distribution(result_df['Signal'], signal_dist, plot_RV, "Signal", 100, status=dist_fix)
    freq_params, og_frequency_samples, frequency_dist, freq_test = fit_frequency_distribution(result_df['Frequency'], frequency_dist, plot_RV, "Frequency", 100, status=dist_fix)
    
    signal_params, og_signal_samples, signal_dist, sig_test = fit_intensity_distribution(result_df['Signal'][result_df['Frequency'] > 0], signal_dist, plot_RV, "Signal", 100, status=dist_fix)
    intensity_params, og_intensity_samples, intensity_dist, int_test = fit_intensity_distribution(result_df['Intensity'][result_df['Frequency'] > 0], intensity_dist, plot_RV, "Intensity", 100, status=dist_fix)
    duration_params, og_duration_samples, duration_dist, dur_test = fit_duration_distribution(result_df['Duration'][result_df['Frequency'] > 0], duration_dist, plot_RV, "Duration", 100, status=dist_fix)
    
    dist_params = {
        'signal_freq': (freq_signal_params, freq_signal_dist),
        'signal': (signal_params, signal_dist),
        'intensity': (intensity_params, intensity_dist),
        'duration': (duration_params, duration_dist),
        'frequency': (freq_params, frequency_dist),
    }

    dist_parameters.append(dist_params)    
    
    dists = pd.DataFrame({
        'Model_Component': list(dist_params.keys()),
        'Model_Evaluation_1': ['KS Test', 'KS Test', 'KS Test', 'Chi-Squared Test', 'Chi-Squared Test'],
        'Evaluation_Value_1': [freq_sig_test[0], sig_test[0], int_test[0], dur_test[0], freq_test[0]],
        'Model_Evaluation_2': ['p_val', 'p_val', 'p_val', 'p_val', 'p_val'],
        'Evaluation_Value_2': [freq_sig_test[1], sig_test[1], int_test[1], dur_test[1], freq_test[1]],
    })
    
    if record_dists:
        dists.to_csv(f"{folder}/Dists_{run_name}_{var}.csv", index=False)

KS Test Statistic: 0.0751164546927004, p-value: 0.0530598943579067
Fit enforced by user.
Chi-Squared Test Statistic: 5.667999063753474, p-value: 0.1289286589576359
Fit enforced by user.
KS Test Statistic: 0.2815851093881819, p-value: 3.438811828043986e-09
Fit enforced by user.
KS Test Statistic: 0.42994347259418153, p-value: 2.4496655980945516e-21
Fit enforced by user.
Chi-Squared Test Statistic: 82.15959596594703, p-value: 4.992769854680295e-15
Fit enforced by user.
KS Test Statistic: 0.19976635000170867, p-value: 5.106290700589333e-12
Fit enforced by user.
Chi-Squared Test Statistic: 3.4829039643871713, p-value: 0.32298619774944043
Fit enforced by user.
KS Test Statistic: 0.38783126812366253, p-value: 9.046001869690178e-22
Fit enforced by user.
KS Test Statistic: 0.49402223983589355, p-value: 6.75759376009247e-36
Fit enforced by user.
Chi-Squared Test Statistic: 100.97243160912811, p-value: 2.7010604929617455e-18
Fit enforced by user.
KS Test Statistic: 0.24771540945210374, p-value: 

## Adapted Neyman-Scott Process storm generation

In [39]:
for i in range(len(target_cities)):
    var = target_cities[i]
    result_df = result_dfs[i]
    multivar_signal = multivar_signals[i]
    cluster_dict = cluster_dicts[i]
    signal_params = dist_parameters[i]['signal']
    intensity_params = dist_parameters[i]['intensity']
    duration_params = dist_parameters[i]['duration']
    freq_params = dist_parameters[i]['frequency']
    freq_signal_params = dist_parameters[i]['signal_freq']
    
    # Assume 'time' is a pandas datetime index or Series
    max_time = np.max(time)  # This should be a Timestamp if time is datetime
    
    # Create future monthly periods
    future_months = pd.date_range(start=max_time + pd.offsets.MonthBegin(1), periods=pred_horizon, freq='MS')
    
    # Repeat 'future_months' for each simulation
    month_column = np.tile(future_months, n_simulations)
    
    # Repeat simulation indices for each time (x times)
    sim_column = np.repeat(np.arange(n_simulations), pred_horizon)
    predicted = np.clip((forecast[:, [i]].flatten() * stds[var]) + means[var], a_min=0, a_max=None)
    signal_column = np.tile(predicted, n_simulations)
        
    # Create the DataFrame
    future_signal = pd.DataFrame({
        'month': month_column,
        'sim': sim_column,
        'signal': signal_column
    })

    if nonstationary_type == 'KNN':
        future_signal = KNN(future_signal, multivar_signal)
        dist_params['signal'] = (signal_params, 'Expon')
        dist_params['signal_freq'] = (freq_signal_params, 'Expon')
        dist_params['intensity'] = (intensity_params, 'Expon')
        dist_params['duration'] = (duration_params, 'Expon')
        print("Nonstationary Signal Parameters Derived Successfully.")
    elif nonstationary_type == 'KNN_MLE':
        future_signal = KNN_MLE(future_signal, multivar_signal)
        dist_params['signal'] = (signal_params, 'Expon')
        dist_params['signal_freq'] = (freq_signal_params, 'Expon')
        dist_params['intensity'] = (intensity_params, 'Expon')
        dist_params['duration'] = (duration_params, 'Expon')
        print("Nonstationary Signal Parameters Derived Successfully.")
    elif nonstationary_type == 'scaled':
        min_sig = min(np.min(result_df['Signal']), np.min(future_signal['signal']))
        max_sig = max(np.max(result_df['Signal']), np.max(future_signal['signal']))
        scaled = (future_signal['signal'] - min_sig) / (max_sig - min_sig) + 0.5
        future_signal['Scale_Sig'] = scaled*signal_params['scale']
        future_signal['Scale_Int'] = scaled*intensity_params['scale']
        future_signal['Scale_Dur'] = scaled*duration_params['scale']
        future_signal['Scale_Freq'] = scaled*freq_params['lambda']
        print("Nonstationary Signal Parameters Derived Successfully.")
    else:
        future_signal['Scale_Sig'] = 1*signal_params['scale']
        future_signal['Scale_Int'] = 1*intensity_params['scale']
        future_signal['Scale_Dur'] = 1*duration_params['scale']
        future_signal['Scale_Freq'] = 1*freq_params['lambda']
        print("Stationary Signal Parameters Derived Successfully.")
    
    if save_params:
        future_signal.to_csv(f"{folder}/Params_{run_name}_{var}.csv", index=False)

    warnings.filterwarnings('ignore')

    all_data = []
    start_time = tm.time()  # Record the start time before the simulation begins
    
    for sim in range(n_simulations):
        fut_sig = future_signal[future_signal['sim'] == sim].reset_index(drop=True)
        FS_parent = simulate_storm_frequencies(sim, future_months, fut_sig, result_df, dist_params, KNN_sampling=KNN_sampling, plot_pair=False, plot_RV=False)
        parent = expand_rows_based_on_frequency(FS_parent, sim)
        FIDS_parent = simulate_storm_statistics(sim, parent, result_df, fut_sig, dist_params, KNN_sampling=KNN_sampling, plot_pair=False, plot_RV=False)
        traj_parent = storm_trajectories(FIDS_parent, bootstrap_curve, cluster_dict_std, plot=plot_traj)
        all_data.append(traj_parent)
        if (sim+1) % 100 == 0:
            end_time = tm.time()  # Record the end time after the simulation ends
            elapsed_time = end_time - start_time  # Calculate the elapsed time
            print(f"Simulation {sim+1} completed out of {n_simulations} in {elapsed_time:.2f} seconds.")
        
    end_time = tm.time()  # Record the end time after the simulation ends
    elapsed_time = end_time - start_time  # Calculate the elapsed time
    print(f"{n_simulations} Simulations Complete for {var}. Run time: {elapsed_time:.2f} seconds.")
    
    # Concatenate all DataFrames in the list at once
    all_sims = pd.concat(all_data, ignore_index=True)
    
    # Convert sim column to integers
    all_sims = all_sims.astype({'sim': 'int'})
    all_sims = all_sims[['sim', 'month', 'storm_index', 'unique_storm_id', 'signal', 'Frequency', 'Intensity', 'Duration', 'storm_day', 'daily_flow']]  # Reorder columns

    # Sort the DataFrame by 'unique_storm_id'
    all_sims = all_sims.sort_values(by='unique_storm_id')
    # Save to CSV
    all_sims.to_csv(f"{folder}/All_Sims_{run_name}_{var}.csv", index=False)

    if plot_allsims:
        # Intensity over all sims
        plot_violin(
            data=all_sims, 
            group_by_columns=['month', 'sim'], 
            y_column='Intensity', 
            title='Maximum Annual Storm Intensities Over Time, All Simulations', 
            y_label='Max Annual Intensity', 
        )
        
        # Duration over all sims
        plot_violin(
            data=all_sims, 
            group_by_columns=['month', 'sim'], 
            y_column='Duration', 
            title='Average Annual Storm Duration Over Time, All Simulations', 
            y_label='Storm Duration (Days)'
        )
        
        
        # Frequency over all sims
        plot_violin(
            data=all_sims, 
            group_by_columns=['month', 'sim'], 
            y_column='Frequency', 
            title='Annual Storm Frequencies Over Time, All Simulations', 
            y_label='Number of Storms'
        )
    
    if plot_onesim:
        plot_storm_intensities(traj_parent, (float(thres[[var]].iloc[0])* stds[var])+means[var], f_10=rl_df['T10'][i], f_100=rl_df['T100'][i])
    
    print(f"All Done with {var}!")

Nonstationary Signal Parameters Derived Successfully.
Simulation 100 completed out of 1000 in 42.89 seconds.
Simulation 200 completed out of 1000 in 86.11 seconds.
Simulation 300 completed out of 1000 in 128.64 seconds.
Simulation 400 completed out of 1000 in 172.00 seconds.
Simulation 500 completed out of 1000 in 215.86 seconds.
Simulation 600 completed out of 1000 in 259.73 seconds.
Simulation 700 completed out of 1000 in 303.00 seconds.
Simulation 800 completed out of 1000 in 347.12 seconds.
Simulation 900 completed out of 1000 in 391.02 seconds.
Simulation 1000 completed out of 1000 in 434.73 seconds.
1000 Simulations Complete for minneapolis. Run time: 434.73 seconds.
All Done with minneapolis!
Nonstationary Signal Parameters Derived Successfully.
Simulation 100 completed out of 1000 in 59.75 seconds.
Simulation 200 completed out of 1000 in 120.42 seconds.
Simulation 300 completed out of 1000 in 180.41 seconds.
Simulation 400 completed out of 1000 in 239.89 seconds.
Simulation 500

KeyboardInterrupt: 